In [ ]:
!pip install geojson

In [ ]:
# An attempt to get opensourced traffic data. But unfortunately the data is not 
# available for location in Lagos.

# !git clone https://github.com/xoolive/traffic
# cd traffic/
# !pip install .

In [ ]:
# Import libraries
import folium
import geojson
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from geojson import Feature, FeatureCollection
from matplotlib.dates import DateFormatter, DayLocator

import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=X_iAA8OcJroSMGUfOUpWb-JyXpwAvsvjlo1DtiOjDcw&tc=U3ERsPZJatSLxg7ej8TGY0uS9HJW8JZs-bXy1MIejjo&cc=dUW_gNqwDLwljM9ZNPl7VRpHKDZtW8GhXF2XH_K3Szo

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWj5--3Pl91CZlLEvOoa8I68aeaZYMJeB5lFCSTOm2NwbQy4FU2-H_Y

Successfully saved authorization token.


In [ ]:
root_dir = '/content/drive/MyDrive/Environmental_covid 19/' # Location where my file directory are located

In [ ]:
# Read Lagos Geometry data and convert into Featurecollection, since our area of interest is Lagos
with open(os.path.join(root_dir, 'Lagos_geometry.geojson')) as f:
    lagos_geojson = geojson.load(f)
lagos_aoi = ee.FeatureCollection(lagos_geojson['features']).geometry()

## Helper Function to Extract staelite imagery form GEE

In [ ]:
# Reduction method for getting and aggregatig raster image data from google eartn engine
# source: 
def create_reduce_region_function(geometry,
                                  reducer=ee.Reducer.mean(),
                                  scale=1000,
                                  crs='EPSG:4326',
                                  bestEffort=True):
  """Creates a region reduction function.

  Creates a region reduction function intended to be used as the input function
  to ee.ImageCollection.map() for reducing pixels intersecting a provided region
  to a statistic for each image in a collection. See ee.Image.reduceRegion()
  documentation for more details.

  Args:
    geometry:
      An ee.Geometry that defines the region over which to reduce data.
    reducer:
      Optional; An ee.Reducer that defines the reduction method.
    scale:
      Optional; A number that defines the nominal scale in meters of the
      projection to work in.
    crs:
      Optional; An ee.Projection or EPSG string ('EPSG:5070') that defines
      the projection to work in.
    maxPixels:
      Optional; A number specifying the maximum number of pixels to reduce.

  Returns:
    A function that accepts an ee.Image and reduces it by region, according to
    the provided arguments.
  """

  def reduce_region_function(img):
    """Applies the ee.Image.reduceRegion() method.

    Args:
      img:
        An ee.Image to reduce to a statistic by region.

    Returns:
      An ee.Feature that contains properties representing the image region
      reduction results per band and the image timestamp formatted as
      milliseconds from Unix epoch (included to enable time series plotting).
    """

    stat = img.reduceRegion(
        reducer=reducer,
        geometry=geometry,
        scale=scale,
        crs=crs)

    return ee.Feature(geometry, stat).set({'millis': img.date().millis()})
  return reduce_region_function


# Define a function to transfer feature properties to a dictionary
def fc_to_dict(fc):
  prop_names = fc.first().propertyNames()
  prop_lists = fc.reduceColumns(
      reducer=ee.Reducer.toList().repeat(prop_names.size()),
      selectors=prop_names).get('list')

  return ee.Dictionary.fromLists(prop_names, prop_lists)

# Function to add date variables to DataFrame.
def add_date_info(df):
  df['Timestamp'] = pd.to_datetime(df['millis'], unit='ms')
  df['Year'] = pd.DatetimeIndex(df['Timestamp']).year
  df['Month'] = pd.DatetimeIndex(df['Timestamp']).month
  df['DOY'] = pd.DatetimeIndex(df['Timestamp']).dayofyear
  return df

In [ ]:
# create dictionary of list of Image collection and image band of interest.
#  This image collection contains environmental values that defines air quality
image_collection = ["COPERNICUS/S5P/OFFL/L3_NO2", 
                    "COPERNICUS/S5P/OFFL/L3_CO", 
                    "COPERNICUS/S5P/OFFL/L3_AER_AI", 
                    "COPERNICUS/S5P/OFFL/L3_SO2"]

band_type = ['tropospheric_NO2_column_number_density', 
             'CO_column_number_density',
             'absorbing_aerosol_index',
             'SO2_column_number_density']

AQ_image_collection_dict = dict(zip(image_collection, band_type))

In [ ]:
# NO2, Carbon_mono_oxide, Aerosol Optical Depth and sulphur oxide
def extract_image_collection(image_collection_id, band_type):
    """
    execute extraction of pixel value from raster image at a 
    specific location

    Arg:
      image_collection_id:
        The id attributed the satelite imagery of interest this value should 
        be gotten from google earth engine

      band_type:
        The band to be used from the stated image_collection_id; it is dependent
        on the image_collection_id

    Returns:
      AQ_stat_fc: GEE feature collection containing image pixel value of the raster
      image specified in the argument 

    """

    
    AQ_dataset = ee.ImageCollection(image_collection_id) \
                      .filter(ee.Filter.calendarRange(2019, 2021, 'year')) \
                      .filter(ee.Filter.calendarRange(1, 8, 'month'))  \
                      .select(band_type) \
                      .filterBounds(lagos_aoi)

    
    # ensure the raster data is limited to the aoi and region reduced using mean
    data_reduced_region = create_reduce_region_function(
                      geometry=lagos_aoi, 
                      reducer=ee.Reducer.mean(),
                      scale=1000)
    # execute the function stated above, for the region specified
    AQ_stat_fc = ee.FeatureCollection(AQ_dataset.map(data_reduced_region)).filter(
      ee.Filter.notNull(AQ_dataset.first().bandNames()))
    
    return AQ_stat_fc

In [ ]:
# loop through the AQ dictionary get the collection value and save the output back in a dictionary
AQ_stat_dict = {}
for collections in AQ_image_collection_dict:

  AQ_stat_fc = extract_image_collection(collections, AQ_image_collection_dict[collections] )
  AQ_stat_dict[collections.split('/')[-1]] =AQ_stat_fc
  
  # on completion, print collection
  print(collections)

COPERNICUS/S5P/OFFL/L3_NO2
COPERNICUS/S5P/OFFL/L3_CO
COPERNICUS/S5P/OFFL/L3_AER_AI
COPERNICUS/S5P/OFFL/L3_SO2


In [ ]:
# extract the data in Feature collection and put them in a dictionary
AQ_extracted_dict = {}
for fc_stat in AQ_stat_dict:
  AQ_extracted_dict[fc_stat] = fc_to_dict(AQ_stat_dict[fc_stat]).getInfo()
  print(fc_stat)

L3_SO2


In [ ]:
# convert dict to dataFrame for each of the data of interest

no2_df = pd.DataFrame(AQ_extracted_dict['L3_NO2'])
co_df = pd.DataFrame(AQ_extracted_dict['L3_CO'])
aer_ai_df = pd.DataFrame(AQ_extracted_dict['L3_AER_AI'])
so2_df = pd.DataFrame(AQ_extracted_dict['L3_SO2'])

# all AQ in dataframe list
AQ_df_list = [no2_df, co_df, aer_ai_df, so2_df]

In [ ]:
# add date information to the dataframes
for df in AQ_df_list:
  add_date_info(df)

In [ ]:
# save all the data in csv into specified drive
for index, AQ_name in enumerate(AQ_extracted_dict):
  
  AQ_df = AQ_df_list[index]
  AQ_extracted_dict[AQ_name]
  file_name = os.path.join(root_dir, f'Sentinel-5P Air Polluant/{AQ_name}_Lagos_JanAug20192021')
  AQ_df.to_csv(file_name, index=False)
  print(index, AQ_name)

0 L3_SO2


# Export GIF image of Night Light data

In [ ]:
# Export night light image before covid
viirs2020_before_convid = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG") \
                .filterDate('2020-01-01','2020-03-31') \
                .select('avg_rad') \
                .median()
lagos_viirs_before_covid = viirs2020_before_convid.clip(lagos_aoi)

task = ee.batch.Export.image.toDrive(image=lagos_viirs_before_covid,
                                     scale=30,
                                     fileFormat='GeoTIFF',
                                     description='lagos_VIIRS_JanMar2020',
                                     folder='Environmental_covid 19',
                                     maxPixels=1e9)
task.start()

In [ ]:
# Export night light image during lockdown
viirs2018_during_lockdown = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG") \
                .filterDate('2020-04-01','2020-05-05') \
                .select('avg_rad') \
                .median()
lagos_viirs_during_lockdown = viirs2018_during_lockdown.clip(lagos_aoi)

task = ee.batch.Export.image.toDrive(image=lagos_viirs_during_lockdown,
                                     scale=30,
                                     fileFormat='GeoTIFF',
                                     description='lagos_VIIRS_AprilMay2020',
                                     folder='Environmental_covid 19',
                                     maxPixels=1e9)
task.start()

# Export GIF Image of Environmental data(Pollutant)

In [ ]:
# Export environmental data
# environmental data similar year before convid
for collection in AQ_image_collection_dict:
  
  
  AQ_before_lockdown = ee.ImageCollection(collection) \
                .filterDate('2019-04-01','2019-05-05') \
                .select(AQ_image_collection_dict[collection]) \
                .median()
  lagos_AQ = AQ_before_lockdown.clip(lagos_aoi)
  
  task = ee.batch.Export.image.toDrive(image=lagos_AQ,
                                     scale=30,
                                     fileFormat='GeoTIFF',
                                     description=f'lagos_{AQ_image_collection_dict[collection]}_AprilMay2019',
                                     folder='Environmental_covid 19',
                                     maxPixels=1e9)
  task.start()

In [ ]:
# environmental data similar year during convid
for collection in AQ_image_collection_dict:
  
  
  AQ_before_lockdown = ee.ImageCollection(collection) \
                .filterDate('2020-04-01','2020-05-05') \
                .select(AQ_image_collection_dict[collection]) \
                .median()
  lagos_AQ = AQ_before_lockdown.clip(lagos_aoi)
  
  task = ee.batch.Export.image.toDrive(image=lagos_AQ,
                                     scale=30,
                                     fileFormat='GeoTIFF',
                                     description=f'lagos_{AQ_image_collection_dict[collection]}_AprilMay2020',
                                     folder='Environmental_covid 19',
                                     maxPixels=1e9)
  task.start()

# Extract NightLight values

In [ ]:
# Export Night light values
night_time_data = extract_image_collection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG', 'avg_rad' )
df_night_time = pd.DataFrame(fc_to_dict(night_time_data).getInfo())

add_date_info(df_night_time)

file_name = os.path.join(root_dir, f'monthly_nightlight_Lagos_JanAug20192021')
df_night_time.to_csv(file_name, index=False)